In [ ]:
# %%capture
# !pip install dlt[duckdb] alive-progress


In [2]:
import dlt


In [4]:
# Question 1
dlt.__version__

'1.6.1'

In [6]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

In [36]:
# Define sources
dlt.source
def dezoomcamp_source():
    client = RESTClient(
        base_url='https://us-central1-dlthub-analytics.cloudfunctions.net',
        paginator=PageNumberPaginator(
            base_page = 1,
            total_path= None
        )
    )
    
    dlt.resource(table_name='rides')
    def rides():
        for page in client.paginate('data_engineering_zoomcamp_api'):
            yield page
            
    return rides
        


In [37]:
# Configure pipeline for extraction, normalizing, and loading
pipeline = dlt.pipeline(
    pipeline_name = 'ny_taxi_pipeline',
    destination = 'duckdb',
    dataset_name= 'ny_taxi_data',
    progress= 'alive_progress'
)

In [38]:
# Run pipeline
load_info = pipeline.run(dezoomcamp_source(), write_disposition="replace")
print(load_info)

2025-02-15 14:58:30,858|[WARNING]|3740965|125654417277056|dlt|configuration.py|_path_from_pipeline:178|Duckdb attached to pipeline ny_taxi_pipeline in path ny_taxi_pipeline.duckdb was could not be found but pipeline has already ran. This may be a result of (1) recreating or attaching pipeline  without or with changed explicit path to database that was used when creating the pipeline. (2) keeping the path to to database in secrets and changing the current working folder so  dlt cannot see them. (3) you deleting the database.


Extract ny_taxi: Resources |████████████████████████████████████████| 1/1 [100%] in 20.4s (0.05/s) 
Normalize ny_taxi in 1739649510.898063: Files |████████████████████████████████████████| 1/1 [100%] in 0.5s (1.97/s) 


2025-02-15 14:58:51,832|[WARNING]|3740965|125654417277056|dlt|configuration.py|_path_from_pipeline:178|Duckdb attached to pipeline ny_taxi_pipeline in path ny_taxi_pipeline.duckdb was could not be found but pipeline has already ran. This may be a result of (1) recreating or attaching pipeline  without or with changed explicit path to database that was used when creating the pipeline. (2) keeping the path to to database in secrets and changing the current working folder so  dlt cannot see them. (3) you deleting the database.


Load ny_taxi in 1739649510.898063: Jobs |████████████████████████████████████████| 1/1 [100%] in 0.8s (1.24/s) 
Pipeline ny_taxi_pipeline load step completed in 0.81 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////media/isaac/DATA/Documents/dezoomcamp/dlt_workshop_homework/ny_taxi_pipeline.duckdb location to store data
Load package 1739649510.898063 is LOADED and contains no failed jobs


In [44]:
# question 2
import duckdb

con = duckdb.connect(f'{pipeline.pipeline_name}.duckdb')

con.sql(f"SET search_path = '{pipeline.dataset_name}'")

con.sql('DESCRIBE').df()

,database,schema,name,column_names,column_types,temporary
0,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,ny_taxi_pipeline,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,ny_taxi_pipeline,ny_taxi_data,rides,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


In [46]:
# Question 3
con.sql('select count(*) from rides')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        10000 │
└──────────────┘

In [47]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

2025-02-15 15:03:26,619|[WARNING]|3740965|125654417277056|dlt|configuration.py|_path_from_pipeline:178|Duckdb attached to pipeline ny_taxi_pipeline in path ny_taxi_pipeline.duckdb was could not be found but pipeline has already ran. This may be a result of (1) recreating or attaching pipeline  without or with changed explicit path to database that was used when creating the pipeline. (2) keeping the path to to database in secrets and changing the current working folder so  dlt cannot see them. (3) you deleting the database.


[(12.3049,)]
